### Initial setup of SEAD staging database

In [43]:
import sys
sys.path = sys.path + ['.'] if '.' not in sys.path else []
import db

# postgres = db.get_db_engine('postgres')
staging = db.get_db_engine('sead_master_9', host='snares.humlab.umu.se')

%load_ext sql_magic
%config SQL.conn_name = 'staging'
%config SQL.output_result = True
%config SQL.notify_result = True


The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


In [44]:
%%read_sql -c staging
select ns.nspname, c.relname, d.description
from pg_class c
join pg_description d
  on d.objoid = c.oid
join pg_namespace ns
  on ns.oid = c.relnamespace
where TRUE
  and c.relkind = 'r'
 -- and ns.nspname = 'public'


Query started at 02:23:11 PM CEST; Query executed in 0.00 m

,nspname,relname,description


In [ ]:
2

CREATE SEQUENCE table_name_id_seq
   OWNED BY table_name.id;

ALTER TABLE table_name
   ALTER id
      SET DEFAULT nextval('table_name_id_seq'::regclass);


        pg_get_serial_sequence(table_name, column_name)
        
CREATE OR REPLACE FUNCTION make_into_serial(table_name TEXT, column_name TEXT) RETURNS INTEGER AS $$
DECLARE
    start_with INTEGER;
    sequence_name TEXT;
BEGIN
    pg_get_serial_sequence(table_name, column_name)
    sequence_name := table_name || '_' || column_name || '_seq';
    EXECUTE 'SELECT coalesce(max(' || column_name || '), 0) + 1 FROM ' || table_name
            INTO start_with;
    EXECUTE 'CREATE SEQUENCE ' || sequence_name ||
            ' START WITH ' || start_with ||
            ' OWNED BY ' || table_name || '.' || column_name;
    EXECUTE 'ALTER TABLE ' || table_name || ' ALTER COLUMN ' || column_name ||
            ' SET DEFAULT nextVal(''' || sequence_name || ''')';
    RETURN start_with;
END;
$$ LANGUAGE plpgsql VOLATILE;    

### Create sead_staging as a copy of sead_master_8
This version of sead_master_8 contains only the public schema.

In [13]:
%%bash
git pull

Updating b1adda6..40f5349
Fast-forward
 utility/sqitch.plan | 3 ---
 1 file changed, 3 deletions(-)


From https://github.com/humlab-sead/sead_change_control
   b1adda6..40f5349  master     -> origin/master


In [20]:
%%bash
dbhost=`cat ~/.default.sead.server`
dbuser=`cat ~/.default.sead.username`

if [ "$dbhost" != "130.239.1.181" ]; then
    echo "This script can for now only be run on 130.239.1.181";
    exit 64
fi

echo "$dbhost"

sql=$(cat <<EOF
    select pg_terminate_backend(pg_stat_activity.pid)
    from pg_stat_activity 
    where pg_stat_activity.datname in ('sead_staging', 'sead_master_9')
      and pid <> pg_backend_pid();
EOF
)

psql --host=$dbhost --username=$dbuser --no-password --dbname=postgres --command "$sql"

sql='drop database if exists sead_staging;'
psql --host=$dbhost --username=$dbuser --no-password --dbname=postgres --command "$sql"

sql='create database sead_staging with template sead_master_9 OWNER sead_master;'
psql --host=$dbhost --username=$dbuser --no-password --dbname=postgres --command "$sql"

130.239.1.181
 pg_terminate_backend 
----------------------
 t
 t
 t
 t
(4 rows)

DROP DATABASE
CREATE DATABASE


### Apply utility change request package

In [21]:
%%bash
#sqitch tag CS_STAGING_20190517_CREATE_DATABASE --tag v0.01 --note "Create staging database" --plan-file ./utility/sqitch.plan
#sqitch tag --tag v0.1 --note "Initial set of CRs" --plan-file ./utility/sqitch.plan

sqitch deploy --target staging --to @v0.01 --mode change --no-verify -C ./utility/
sqitch deploy --target staging --to @v0.1 --mode change --no-verify -C ./utility/
    

Adding registry tables to staging
Deploying changes through 20190517_DDL_STAGING_CREATE_DATABASE @v0.01 to staging
  + 20190517_DDL_STAGING_CREATE_DATABASE @v0.01 .. ok
Deploying changes through 20190411_DDL_UTILITY_ENABLE_PGCRYPTO @v0.1 to staging
  + 20190407_DDL_UTILITY_CREATE_UTILITY_SCHEMA ......... ok
  + 20190407_DDL_UTILITY_CREATE_TABLE_DEPENDENCY_VIEW .. ok
  + 20190407_DDL_UTILITY_CONVERT_SEQUENCES_TO_SERIAL ... ok
  + 20190407_DDL_UTILITY_SYNC_ALL_SEQUENCES ............ ok
  + 20190407_DDL_UTILITY_ADD_UUID_SUPPORT .............. ok
  + 20190411_DDL_UTILITY_ENABLE_TABLEFUNC .............. ok
  + 20190411_DDL_UTILITY_ENABLE_PGCRYPTO @v0.1 ......... ok


### Apply general change request package

In [26]:
%%bash
#sqitch tag 20170101_DDL_BIBLIO_REFACTOR_MODEL --tag v0.1 --note "Staging DB with initial DDL changes" --plan-file ./general/sqitch.plan
#sqitch tag 20190513_DML_ECOCODE_ADD_GROUP_UPDATE_SYSTEM --tag v0.2--note "Staging DB with initial DML updates" --plan-file ./general/sqitch.plan

# sqitch deploy --target staging --to @v0.1 --mode change --no-verify -C ./general/
sqitch deploy --target staging --to @v0.2 --mode change --no-verify -C ./general/
    

Deploying changes through 20190513_DML_ECOCODE_ADD_GROUP_UPDATE_SYSTEM @v0.2 to staging
  + 20180521_DML_DENDRO_ADD_LOOKUP_DATA ................. not ok


psql:deploy/20180521_DML_DENDRO_ADD_LOOKUP_DATA.sql:168: ERROR:  value too long for type character varying(40)
"/usr/bin/psql" unexpectedly returned exit value 3



CalledProcessError: Command 'b'#sqitch tag 20170101_DDL_BIBLIO_REFACTOR_MODEL --tag v0.1 --note "Staging DB with initial DDL changes" --plan-file ./general/sqitch.plan\n#sqitch tag 20190513_DML_ECOCODE_ADD_GROUP_UPDATE_SYSTEM --tag v0.2--note "Staging DB with initial DML updates" --plan-file ./general/sqitch.plan\n\n# sqitch deploy --target staging --to @v0.1 --mode change --no-verify -C ./general/\nsqitch deploy --target staging --to @v0.2 --mode change --no-verify -C ./general/\n    \n'' returned non-zero exit status 2

In [18]:
%%bash
sqitch help deploy

sqitch deploy --target staging --to v0.1 --mode change --no-verify -C ./utility

Name
    sqitch-deploy - Deploy changes to a database

Synopsis
      sqitch deploy [options] [<database>]
      sqitch deploy [options] [<database>] <change>
      sqitch deploy [options] [<database>] --to-change <change>

Description
    Deploy changes to the database. Changes will begin from the current
    deployment state. They will run to the latest change, unless a change is
    specified, either via "--to" or with no option flag, in which case changes
    will be deployed up-to and including that change.

    If the database it up-to-date or already deployed to the specified change,
    no changes will be made. If the change appears earlier in the plan than
    the currently-deployed state, an error will be returned, along with a
    suggestion to instead use sqitch-revert.

    The "<database>" parameter specifies the database to which to connect, and
    may also be specified as the "--target" option. It can be target name, a
    URI, an engine name, or plan file path.

Optio

stty: 'standard input': Inappropriate ioctl for device


In [9]:
%%read_sql -c postgres
create database sead_staging with template sead_master_9 OWNER sead_master;

Query started at 12:21:32 PM CEST

InternalError: (psycopg2.errors.ActiveSqlTransaction) CREATE DATABASE cannot run inside a transaction block

[SQL: create database sead_staging with template sead_master_9 OWNER sead_master;]
(Background on this error at: http://sqlalche.me/e/2j85)

In [1]:

print('hej')

hej
